In [1]:
import os
import json
import numpy as np
import cv2
from tqdm import tqdm
import kagglehub
from ultralytics import FastSAM
from PIL import Image

c:\Users\admin\OneDrive - National Economics University\Desktop\Các Folder\Label\auto_label_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\admin\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# 1️.Tải dataset từ Kaggle
path = kagglehub.dataset_download("pavansanagapati/images-dataset")
print("Dataset path:", path)

100%|██████████| 1.01G/1.01G [14:41<00:00, 1.24MB/s]

Extracting files...


Dataset path: C:\Users\admin\.cache\kagglehub\datasets\pavansanagapati\images-dataset\versions\1


In [3]:
# 2️.Load model segmentation tự động
model = FastSAM('FastSAM-x.pt')  
# model nhẹ, chạy CPU được

In [4]:
# 3️.Định nghĩa thư mục chứa ảnh
root_dir = os.path.join(path, "data")  
# Dataset có cấu trúc /data/<class>/
output_dir = os.path.join(path, "labels_auto")
os.makedirs(output_dir, exist_ok=True)

In [5]:
# 4️.Hàm convert mask -> polygon (Labelme format)
def mask_to_polygon(mask):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for contour in contours:
        contour = contour.squeeze()
        if len(contour.shape) == 2:
            polygons.append(contour.flatten().tolist())
    return polygons

In [6]:
# 5️. Duyệt qua tất cả thư mục con (class)
for cls_name in os.listdir(root_dir):
    cls_path = os.path.join(root_dir, cls_name)
    if not os.path.isdir(cls_path):
        continue
    
    output_cls = os.path.join(output_dir, cls_name)
    os.makedirs(output_cls, exist_ok=True)
    
    print(f"\n Processing class: {cls_name}")
    
    for img_name in tqdm(os.listdir(cls_path)):
        if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        
        img_path = os.path.join(cls_path, img_name)
        image = Image.open(img_path)
        
        # 6️. Chạy segmentation tự động
        results = model(img_path, retina_masks=True, imgsz=512, device='cpu')
        masks = results[0].masks.data.cpu().numpy() if results[0].masks is not None else []
        
        # 7️. Chuyển mask thành polygon và lưu Labelme .json
        shapes = []
        for mask in masks:
            polygon_points = mask_to_polygon(mask)
            for pts in polygon_points:
                shapes.append({
                    "label": cls_name,
                    "points": np.array(pts).reshape(-1, 2).tolist(),
                    "group_id": None,
                    "shape_type": "polygon",
                    "flags": {}
                })
        
        labelme_json = {
            "version": "5.3.1",
            "flags": {},
            "shapes": shapes,
            "imagePath": img_name,
            "imageData": None,
            "imageHeight": image.height,
            "imageWidth": image.width
        }
        
        json_path = os.path.join(output_cls, img_name.replace('.bmp', '.json').replace('.jpg', '.json'))
        with open(json_path, 'w') as f:
            json.dump(labelme_json, f, indent=2)
        
print("\n Hoàn tất tạo nhãn segmentation tự động theo format Labelme!")
print(f" Tất cả file .json nằm trong: {output_dir}")


 Processing class: bike


  0%|          | 0/365 [00:00<?, ?it/s]


image 1/1 C:\Users\admin\.cache\kagglehub\datasets\pavansanagapati\images-dataset\versions\1\data\bike\bike_001.bmp: 384x512 31 objects, 1698.9ms
Speed: 97.3ms preprocess, 1698.9ms inference, 124.5ms postprocess per image at shape (1, 3, 384, 512)


  0%|          | 1/365 [00:07<43:07,  7.11s/it]


image 1/1 C:\Users\admin\.cache\kagglehub\datasets\pavansanagapati\images-dataset\versions\1\data\bike\bike_002.bmp: 384x512 14 objects, 1466.9ms
Speed: 57.4ms preprocess, 1466.9ms inference, 35.0ms postprocess per image at shape (1, 3, 384, 512)


...


image 1/1 C:\Users\admin\.cache\kagglehub\datasets\pavansanagapati\images-dataset\versions\1\data\human\rider-98.jpg: 512x288 14 objects, 1052.7ms
Speed: 1.7ms preprocess, 1052.7ms inference, 12.1ms postprocess per image at shape (1, 3, 512, 288)


100%|█████████▉| 201/202 [05:16<00:01,  1.69s/it]


image 1/1 C:\Users\admin\.cache\kagglehub\datasets\pavansanagapati\images-dataset\versions\1\data\human\rider-99.jpg: 512x512 18 objects, 1923.8ms
Speed: 3.2ms preprocess, 1923.8ms inference, 12.8ms postprocess per image at shape (1, 3, 512, 512)


100%|██████████| 202/202 [05:18<00:00,  1.58s/it]


 Hoàn tất tạo nhãn segmentation tự động theo format Labelme!
 Tất cả file .json nằm trong: C:\Users\admin\.cache\kagglehub\datasets\pavansanagapati\images-dataset\versions\1\labels_auto


**Chuyển dữ liệu sang định dạng để huấn luyện được với YOLO model**

In [8]:
import os
import json
import shutil
import random
from tqdm import tqdm
from PIL import Image

# === Cấu hình ===
LABELME_DIR = "C:/Users/admin/.cache/kagglehub/datasets/pavansanagapati/images-dataset/versions/1/labels_auto"
YOLO_DIR = "C:/Users/admin/OneDrive - National Economics University/Desktop/Các Folder/Label/dataset_yolo"
SPLIT_RATIO = 0.8  # train 80%, val 20%
CLASSES = sorted(os.listdir(LABELME_DIR))

# === Tạo cấu trúc YOLO dataset ===
os.makedirs(f"{YOLO_DIR}/images/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/images/val", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/val", exist_ok=True)

print("Classes:", CLASSES)

Classes: ['bike', 'cars', 'cats', 'data', 'dogs', 'flowers', 'horses', 'human']


In [9]:
# === Hàm chuyển polygon sang YOLO segmentation ===
def polygon_to_yolo(points, width, height):
    yolo_points = []
    for (x, y) in points:
        yolo_points.append(x / width)
        yolo_points.append(y / height)
    return yolo_points

# === Duyệt qua từng class ===
for class_id, cls_name in enumerate(CLASSES):
    cls_path = os.path.join(LABELME_DIR, cls_name)
    imgs = [f for f in os.listdir(cls_path) if f.endswith('.json')]
    random.shuffle(imgs)
    split_idx = int(len(imgs) * SPLIT_RATIO)

    for i, json_file in enumerate(tqdm(imgs, desc=f"Converting {cls_name}")):
        split = "train" if i < split_idx else "val"
        json_path = os.path.join(cls_path, json_file)
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        img_name = data["imagePath"]
        img_path = os.path.join(LABELME_DIR, cls_name, img_name)
        
        # Copy image sang thư mục YOLO
        dst_img_path = os.path.join(YOLO_DIR, f"images/{split}/{cls_name}_{img_name}")
        if os.path.exists(img_path):
            shutil.copy(img_path, dst_img_path)
        
        # Ghi file label .txt
        txt_path = os.path.join(YOLO_DIR, f"labels/{split}/{cls_name}_{img_name.replace('.jpg','.txt').replace('.png','.txt').replace('.bmp','.txt')}")
        width, height = data["imageWidth"], data["imageHeight"]

        with open(txt_path, 'w') as f_txt:
            for shape in data["shapes"]:
                pts = shape["points"]
                yolo_pts = polygon_to_yolo(pts, width, height)
                pts_str = " ".join(map(str, yolo_pts))
                f_txt.write(f"{class_id} {pts_str}\n")

print("\n Đã convert toàn bộ Labelme JSON ➜ YOLOv12 segmentation format!")

Converting cats: 100%|██████████| 202/202 [00:03<00:00, 54.34it/s]
Converting data: 0it [00:00, ?it/s]
Converting dogs: 100%|██████████| 202/202 [00:03<00:00, 51.09it/s]
Converting flowers: 0it [00:00, ?it/s]
Converting human: 100%|██████████| 202/202 [00:03<00:00, 52.63it/s]


 Đã convert toàn bộ Labelme JSON ➜ YOLOv12 segmentation format!


In [ ]:
# === Tạo file data.yaml ===
yaml_content = f"""train: {YOLO_DIR}/images/train
val: {YOLO_DIR}/images/val

nc: {len(CLASSES)}
names: {CLASSES}
"""
with open(os.path.join(YOLO_DIR, "data.yaml"), "w") as f:
    f.write(yaml_content)

print(" Tạo file data.yaml thành công!")
print(f" Dataset YOLO lưu tại: {YOLO_DIR}")